# Task 2: Multi-target single-camera (MTSC) tracking

In [1]:
import os
import gc
import bz2
import pickle
import _pickle as cPickle
import torch
import cv2
import numpy as np
from PIL import Image
from VehicleDetection import *
from itertools import chain
from tqdm import tqdm
from pqdm.processes import pqdm


# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# Import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2_dataset_loader import *


In [2]:
# PATHS
DATASET = "../datasets/aic19-track1-mtmc-train/train/"
SEQUENCES = [DATASET+seq+"/" for seq in os.listdir(DATASET)]
CAMERAS = [[seq+cam+"/" for cam in os.listdir(seq)]for seq in SEQUENCES]
SEQUENCES = [seq.replace(DATASET, "").replace("/", "") for seq in SEQUENCES]
CAMERAS = dict(zip(SEQUENCES, CAMERAS))

# DEFINE SPLITS
train = ["S01", "S04"]
test = ["S04"]

# Model Parameters
selected_model = 'COCO-Detection/retinanet_R_101_FPN_3x.yaml'

In [3]:
def extract_video(path, div_frames, skip):
    vidcap = cv2.VideoCapture(path)
    fps = int(vidcap.get(cv2.CAP_PROP_FPS))
    num_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []
    # Read Half the frames 
    for _ in range(num_frames//div_frames):
        for i in range(skip):
            frame = vidcap.read()[1]
            if i == 0:
                frames.append(frame.astype(np.float32)) # Reduce soze
    return iter(frames) # Iterator

def readDetections(path):
  #Generates detection dictionary where the frame number is the key and the values are the info of the corresponding detection/s
  
    with open(path) as f:
        lines = f.readlines()

    detections = {}
    for line in lines:
        data = line.split(',')
        if data[0] in detections:
            detections[data[0]].append(VehicleDetection(int(data[0]), int(data[1]), float(data[2]), float(data[3]), float(data[4]), float(data[5]), float(data[6])))
        else:
            detections[data[0]] = [VehicleDetection(int(data[0]), int(data[1]), float(data[2]), float(data[3]), float(data[4]), float(data[5]), float(data[6]))]

    return detections


In [4]:
seq_data = []

# For each training seq move through cameras and extact even frames and even gt
for i, seq in enumerate(train):
    for j, cam in tqdm(enumerate(CAMERAS[seq]), total = len(CAMERAS[seq]), desc = f"Processing {seq}..."):
        data = {}
        data["div"] = 1
        data["base_path"] = cam + "frames/" # To Save Frames
        data["gt_detected"] = readDetections(cam + "gt/gt.txt")
        data["gt_detected"] = {key:data["gt_detected"][key] for key in data["gt_detected"].keys() if int(key) % data["div"] == 0}
        data["frames"] = extract_video(cam + "vdo.avi", 4,data["div"])
        seq_data.append(data)
        break
    break

Processing S01...:   0%|          | 0/5 [00:04<?, ?it/s]


In [5]:
def get_dataset():
    global seq_data
    return [get_AICity_dicts(**d) for d in seq_data] # Get List of Dicts in proper format



In [6]:
# Register Dataset
DatasetCatalog.register("AICity_train" , lambda d=seq_data[0]: get_AICity_dicts(**d))
MetadataCatalog.get("AICity_train").set(thing_classes=["car"])
AICity_metadata = MetadataCatalog.get("AICity_train")

gc.collect()

0

In [7]:
# Training
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(selected_model))
cfg.DATASETS.TRAIN = ("AICity_train",)
#cfg.DATASETS.VAL = ('AICity_valid',)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(selected_model)  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 1e-3
cfg.SOLVER.MAX_ITER = 5000
cfg.SOLVER.STEPS = [] # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 # (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.MODEL.BACKBONE.FREEZE_AT = 1

cfg.OUTPUT_DIR = "./results"

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

Loading config /home/yusepp/anaconda3/envs/m5/lib/python3.8/site-packages/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


[04/06 03:07:11 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2)

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


[04/06 03:07:42 d2.engine.train_loop]: Starting training from iteration 0
[04/06 03:07:53 d2.utils.events]:  eta: 0:29:10  iter: 19  total_loss: 2.452  loss_cls: 1.833  loss_box_reg: 0.621  time: 0.3427  data_time: 0.0306  lr: 1.9981e-05  max_mem: 5386M
[04/06 03:08:00 d2.utils.events]:  eta: 0:29:09  iter: 39  total_loss: 1.2  loss_cls: 0.6067  loss_box_reg: 0.549  time: 0.3437  data_time: 0.0029  lr: 3.9961e-05  max_mem: 5386M
[04/06 03:08:07 d2.utils.events]:  eta: 0:28:55  iter: 59  total_loss: 0.7328  loss_cls: 0.2995  loss_box_reg: 0.4447  time: 0.3416  data_time: 0.0030  lr: 5.9941e-05  max_mem: 5386M
[04/06 03:08:14 d2.utils.events]:  eta: 0:28:54  iter: 79  total_loss: 0.5423  loss_cls: 0.207  loss_box_reg: 0.3361  time: 0.3429  data_time: 0.0031  lr: 7.9921e-05  max_mem: 5386M
[04/06 03:08:21 d2.utils.events]:  eta: 0:28:50  iter: 99  total_loss: 0.5068  loss_cls: 0.1797  loss_box_reg: 0.3247  time: 0.3445  data_time: 0.0029  lr: 9.9901e-05  max_mem: 5386M
[04/06 03:08:27 d2.

KeyboardInterrupt: 